In [148]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
import category_encoders as ce
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier 
from sklearn import tree

In [149]:
df = pd.read_csv(r'../data/fake_job_postings.csv')
df.sample(5)

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
9773,9774,Commercial Moving Staff,"US, WI, Milwaukee",NaN,NaN,Schroeder Solutions is a full-service interior...,Schroeder Solutions is a full-service interior...,Experience in the moving or furniture industry...,Schroeder Solutions offers a competitive salar...,0,1,0,Full-time,Entry level,High School or equivalent,NaN,NaN,0
4654,4655,Part Time Creative Project Manager,"US, TN, Nashville",NaN,NaN,NoSleepForSheep is a Nashville-based creative ...,Are you passionate about making people happy a...,1+ year project management experienceException...,Flexible schedule.Opportunity to work with a s...,0,1,1,NaN,NaN,NaN,NaN,NaN,0
10830,10831,UI Developer,"EG, C, Nasrcity",NaN,NaN,Thanks for visiting our Job Board. Please revi...,Main duties and responsibilities:The UI Develo...,"Experience in designing, marking up, developin...",Attractive pay prepackagesMobile allowanceTran...,0,1,0,NaN,NaN,NaN,NaN,NaN,0
14769,14770,Freelance Academic Writer,"US, ME,",NaN,7-12000,The benefits you enjoy working as a freelance ...,Freelance Academic Writer We are hiring freel...,"Please, email us with a subject: ""Freelance wr...",The main benefit of working at UVOCorp agency ...,1,1,0,Other,Not Applicable,Some College Coursework Completed,Writing and Editing,Writing/Editing,0
504,505,Financial Controller,"GB, LND, London",FIN,NaN,Depop is an exciting new mobile marketplace wh...,About DepopDepop is an exciting new mobile mar...,"About YouA good degree is important, ideally (...","Sharing winsEvery day we have standups, every ...",0,1,0,Full-time,Associate,Professional,Accounting,Accounting/Auditing,0


In [150]:
df.shape

(17880, 18)

In [151]:
df.isna().sum()

job_id                     0
title                      0
location                 346
department             11547
salary_range           15012
company_profile         3308
description                1
requirements            2695
benefits                7210
telecommuting              0
has_company_logo           0
has_questions              0
employment_type         3471
required_experience     7050
required_education      8105
industry                4903
function                6455
fraudulent                 0
dtype: int64

In [152]:
df.describe()

,job_id,telecommuting,has_company_logo,has_questions,fraudulent
count,17880.000000,17880.000000,17880.000000,17880.000000,17880.000000
mean,8940.500000,0.042897,0.795302,0.491723,0.048434
std,5161.655742,0.202631,0.403492,0.499945,0.214688
min,1.000000,0.000000,0.000000,0.000000,0.000000
25%,4470.750000,0.000000,1.000000,0.000000,0.000000
50%,8940.500000,0.000000,1.000000,0.000000,0.000000
75%,13410.250000,0.000000,1.000000,1.000000,0.000000
max,17880.000000,1.000000,1.000000,1.000000,1.000000


In [153]:
df.corr()

,job_id,telecommuting,has_company_logo,has_questions,fraudulent
job_id,1.000000,-0.004559,-0.014539,-0.087025,0.079872
telecommuting,-0.004559,1.000000,-0.019836,0.020345,0.034523
has_company_logo,-0.014539,-0.019836,1.000000,0.233932,-0.261971
has_questions,-0.087025,0.020345,0.233932,1.000000,-0.091627
fraudulent,0.079872,0.034523,-0.261971,-0.091627,1.000000


*Decision*  
  
I have decided to completely drop the column for salary range since there is an overwhelming amount of NaN values and if I were to make a decision on how to fill them then I would be introducing a lot of bias in the data.

But before that let's check if there is a correlation between having no value for salary and fraudulent.

In [154]:
df.insert(17, "has_salary", 1)
df.loc[df['salary_range'].isna(), 'has_salary'] = 0

In [155]:
df.corr()

,job_id,telecommuting,has_company_logo,has_questions,has_salary,fraudulent
job_id,1.000000,-0.004559,-0.014539,-0.087025,0.009231,0.079872
telecommuting,-0.004559,1.000000,-0.019836,0.020345,0.071431,0.034523
has_company_logo,-0.014539,-0.019836,1.000000,0.233932,-0.058141,-0.261971
has_questions,-0.087025,0.020345,0.233932,1.000000,0.005103,-0.091627
has_salary,0.009231,0.071431,-0.058141,0.005103,1.000000,0.059696
fraudulent,0.079872,0.034523,-0.261971,-0.091627,0.059696,1.000000


In [156]:
df.drop(columns='salary_range', inplace=True)

In [157]:
df.isna().sum()

job_id                     0
title                      0
location                 346
department             11547
company_profile         3308
description                1
requirements            2695
benefits                7210
telecommuting              0
has_company_logo           0
has_questions              0
employment_type         3471
required_experience     7050
required_education      8105
industry                4903
function                6455
has_salary                 0
fraudulent                 0
dtype: int64

*Decision*  
  
I am going to fill the department NaN values with "Unspecified". Even though there are a lot of values missing I think that it could be potentially less likely for jobs that specify a department to be fraudulent. I could either treat this variable as binary ("has_department", "has_no_department"), or use "Unspecified" and then see if there is maybe another department name that is used for fraudulent jobs.

In [158]:
df['department'] = df['department'].fillna("Unspecified")

In [159]:
df.head()

,job_id,title,location,department,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,has_salary,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",Unspecified,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0,0
4,5,Bill Review Manager,"US, FL, Fort Worth",Unspecified,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0,0


In [160]:
df['department'].nunique()

1338

Let's see if we can make departments less unique

In [161]:
df['department'] = df['department'].str.lower()

In [162]:
df.head()

,job_id,title,location,department,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,has_salary,fraudulent
0,1,Marketing Intern,"US, NY, New York",marketing,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",success,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",unspecified,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0,0
3,4,Account Executive - Washington DC,"US, DC, Washington",sales,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0,0
4,5,Bill Review Manager,"US, FL, Fort Worth",unspecified,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0,0


In [163]:
df.loc[df['department'].str.contains('tech'), 'department'] = 'tech'
df.loc[df['department'].str.contains('it'), 'department'] = 'tech'
df.loc[df['department'].str.contains('develop'), 'department'] = 'tech'
df.loc[df['department'].str.contains('data'), 'department'] = 'tech'
df.loc[df['department'].str.contains('cs'), 'department'] = 'tech'
df.loc[df['department'].str.contains('computer'), 'department'] = 'tech'
df.loc[df['department'].str.contains('sales'), 'department'] = 'sales'
df.loc[df['department'].str.contains('eng'), 'department'] = 'engineering'
df.loc[df['department'].str.contains('client'), 'department'] = 'customer'
df.loc[df['department'].str.contains('pr'), 'department'] = 'customer'
df.loc[df['department'].str.contains('relations'), 'department'] = 'customer'
df.loc[df['department'].str.contains('customer'), 'department'] = 'customer'
df.loc[df['department'].str.contains('business'), 'department'] = 'business'
df.loc[df['department'].str.contains('financ'), 'department'] = 'finance'
df.loc[df['department'].str.contains('bank'), 'department'] = 'finance'
df.loc[df['department'].str.contains('account'), 'department'] = 'accounting'
df.loc[df['department'].str.contains('market'), 'department'] = 'marketing'

In [164]:
df['department'].nunique()

790

We have halved the departments, maybe we can work with that

In [165]:
df['title'].nunique()

11231

Maybe apply the same to titles

In [166]:
df['title'] = df['title'].str.lower()

In [167]:
df.loc[df['title'].str.contains('tech'), 'title'] = 'tech'
df.loc[df['title'].str.contains('it'), 'title'] = 'tech'
df.loc[df['title'].str.contains('develop'), 'title'] = 'tech'
df.loc[df['title'].str.contains('data'), 'title'] = 'tech'
df.loc[df['title'].str.contains('cs'), 'title'] = 'tech'
df.loc[df['title'].str.contains('computer'), 'title'] = 'tech'
df.loc[df['title'].str.contains('sales'), 'title'] = 'sales'
df.loc[df['title'].str.contains('eng'), 'title'] = 'engineering'
df.loc[df['title'].str.contains('client'), 'title'] = 'customer'
df.loc[df['title'].str.contains('pr'), 'title'] = 'customer'
df.loc[df['title'].str.contains('relations'), 'title'] = 'customer'
df.loc[df['title'].str.contains('customer'), 'title'] = 'customer'
df.loc[df['title'].str.contains('business'), 'title'] = 'business'
df.loc[df['title'].str.contains('financ'), 'title'] = 'finance'
df.loc[df['title'].str.contains('bank'), 'title'] = 'finance'
df.loc[df['title'].str.contains('account'), 'title'] = 'accounting'
df.loc[df['title'].str.contains('market'), 'title'] = 'marketing'
df.loc[df['title'].str.contains('manager'), 'title'] = 'manager'

In [168]:
df['title'].nunique()

3233

Let's recap our na values

In [169]:
df.isna().sum()

job_id                    0
title                     0
location                346
department                0
company_profile        3308
description               1
requirements           2695
benefits               7210
telecommuting             0
has_company_logo          0
has_questions             0
employment_type        3471
required_experience    7050
required_education     8105
industry               4903
function               6455
has_salary                0
fraudulent                0
dtype: int64

*Decision*  
  
I am going to fill the rows that give no location with none.

In [170]:
df['location'] = df['location'].fillna("none,")

In [171]:
df.isna().sum()

job_id                    0
title                     0
location                  0
department                0
company_profile        3308
description               1
requirements           2695
benefits               7210
telecommuting             0
has_company_logo          0
has_questions             0
employment_type        3471
required_experience    7050
required_education     8105
industry               4903
function               6455
has_salary                0
fraudulent                0
dtype: int64

Now I am going to process the location. Maybe just use the country of the location, and put in state as a second column, the existence of a state or even the state itself could be an indicator

In [172]:
df['location'] = df['location'].str.lower()
df['country'] = df[["location"]].applymap(lambda str:str.split(",")[0] if "," in str else str)
df['state'] = df[["location"]].applymap(lambda str:str.split(",")[1] if "," in str else 'none')
df.loc[df['state'] == '', 'state'] = 'none'

In [173]:
df.head()

,job_id,title,location,department,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,has_salary,fraudulent,country,state
0,1,marketing,"us, ny, new york",marketing,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0,0,us,ny
1,2,customer,"nz, , auckland",success,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0,0,nz,
2,3,commissioning machinery assistant (cma),"us, ia, wever",unspecified,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0,0,us,ia
3,4,accounting,"us, dc, washington",sales,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0,0,us,dc
4,5,manager,"us, fl, fort worth",unspecified,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0,0,us,fl


In [174]:
column_to_move = df.pop("country")
df.insert(3, "country", column_to_move)
column_to_move = df.pop("state")
df.insert(4, "state", column_to_move)
df.head()

,job_id,title,location,country,state,department,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,has_salary,fraudulent
0,1,marketing,"us, ny, new york",us,ny,marketing,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0,0
1,2,customer,"nz, , auckland",nz,,success,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0,0
2,3,commissioning machinery assistant (cma),"us, ia, wever",us,ia,unspecified,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0,0
3,4,accounting,"us, dc, washington",us,dc,sales,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0,0
4,5,manager,"us, fl, fort worth",us,fl,unspecified,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0,0


In [175]:
df.isna().sum()

job_id                    0
title                     0
location                  0
country                   0
state                     0
department                0
company_profile        3308
description               1
requirements           2695
benefits               7210
telecommuting             0
has_company_logo          0
has_questions             0
employment_type        3471
required_experience    7050
required_education     8105
industry               4903
function               6455
has_salary                0
fraudulent                0
dtype: int64

In [176]:
df['country'].nunique()

91

In [177]:
df['state'].nunique()

326

I am going to just drop the location column now and use the country and state instead.

In [178]:
df.drop(columns=['location'], inplace=True)

*Decision*  
  
I will filter the requirements column and see if some keywords appear that I would expect in a real job posting. Words like "experience", "years", "requirements", "qualifications", "must", "degree", "bachelor", "skill". I will then use a binary column to indicate whether one or multiple of these words were present in the description.

In [179]:
df.insert(7, "has_reqs", 0)

In [180]:
df.loc[df['requirements'].str.contains('exp') | df['requirements'].str.contains('year') | df['requirements'].str.contains('qualif') |
       df['requirements'].str.contains('must') | df['requirements'].str.contains('degree') | df['requirements'].str.contains('bachelor') |
       df['requirements'].str.contains('master') | df['requirements'].str.contains('skill'), 'has_reqs'] = 1

In [181]:
df.corr()

,job_id,has_reqs,telecommuting,has_company_logo,has_questions,has_salary,fraudulent
job_id,1.000000,-0.043293,-0.004559,-0.014539,-0.087025,0.009231,0.079872
has_reqs,-0.043293,1.000000,-0.016096,0.240892,0.185417,0.017069,-0.062734
telecommuting,-0.004559,-0.016096,1.000000,-0.019836,0.020345,0.071431,0.034523
has_company_logo,-0.014539,0.240892,-0.019836,1.000000,0.233932,-0.058141,-0.261971
has_questions,-0.087025,0.185417,0.020345,0.233932,1.000000,0.005103,-0.091627
has_salary,0.009231,0.017069,0.071431,-0.058141,0.005103,1.000000,0.059696
fraudulent,0.079872,-0.062734,0.034523,-0.261971,-0.091627,0.059696,1.000000


*Observation*  
  
It seems like jobs that have no requirements or benefits might mention them in company profile or description, so I will implement the same logic there with the same words for has_reqs.

In [182]:
df.loc[df['company_profile'].str.contains('exp') | df['company_profile'].str.contains('year') | df['company_profile'].str.contains('qualif') |
       df['company_profile'].str.contains('must') | df['company_profile'].str.contains('degree') | df['company_profile'].str.contains('bachelor') |
       df['company_profile'].str.contains('master') | df['company_profile'].str.contains('skill'), 'has_reqs'] = 1

df.loc[df['description'].str.contains('exp') | df['description'].str.contains('year') | df['description'].str.contains('qualif') |
       df['description'].str.contains('must') | df['description'].str.contains('degree') | df['description'].str.contains('bachelor') |
       df['description'].str.contains('master') | df['description'].str.contains('skill'), 'has_reqs'] = 1

In [183]:
df.corr()

,job_id,has_reqs,telecommuting,has_company_logo,has_questions,has_salary,fraudulent
job_id,1.000000,-0.006999,-0.004559,-0.014539,-0.087025,0.009231,0.079872
has_reqs,-0.006999,1.000000,-0.009678,0.137564,0.038413,-0.006388,-0.081021
telecommuting,-0.004559,-0.009678,1.000000,-0.019836,0.020345,0.071431,0.034523
has_company_logo,-0.014539,0.137564,-0.019836,1.000000,0.233932,-0.058141,-0.261971
has_questions,-0.087025,0.038413,0.020345,0.233932,1.000000,0.005103,-0.091627
has_salary,0.009231,-0.006388,0.071431,-0.058141,0.005103,1.000000,0.059696
fraudulent,0.079872,-0.081021,0.034523,-0.261971,-0.091627,0.059696,1.000000


We managed to achieve higher correlation this way between has_reqs and fraudulent.

*Decision*  
  
Let's maybe see if there is correlation between has_industry and fraudulent.

In [184]:
df.insert(17, "has_industry", 1)
df.loc[df['industry'].isna(), 'has_industry'] = 0

In [185]:
df.corr()

,job_id,has_reqs,telecommuting,has_company_logo,has_questions,has_industry,has_salary,fraudulent
job_id,1.000000,-0.006999,-0.004559,-0.014539,-0.087025,0.013633,0.009231,0.079872
has_reqs,-0.006999,1.000000,-0.009678,0.137564,0.038413,0.060417,-0.006388,-0.081021
telecommuting,-0.004559,-0.009678,1.000000,-0.019836,0.020345,0.021237,0.071431,0.034523
has_company_logo,-0.014539,0.137564,-0.019836,1.000000,0.233932,0.137759,-0.058141,-0.261971
has_questions,-0.087025,0.038413,0.020345,0.233932,1.000000,0.055148,0.005103,-0.091627
has_industry,0.013633,0.060417,0.021237,0.137759,0.055148,1.000000,0.223232,-0.021915
has_salary,0.009231,-0.006388,0.071431,-0.058141,0.005103,0.223232,1.000000,0.059696
fraudulent,0.079872,-0.081021,0.034523,-0.261971,-0.091627,-0.021915,0.059696,1.000000


The correlation is minimal, I would probably say there is none, but I will keep it as a feature for now.

In [186]:
df.loc[df['fraudulent']==1].sample(20)

,job_id,title,country,state,department,company_profile,description,has_reqs,requirements,benefits,...,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,has_industry,function,has_salary,fraudulent
14270,14271,tech,none,none,unspecified,NaN,build a website,0,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,0,NaN,0,1
17602,17603,marketing,us,nh,unspecified,NaN,"Are you looking to make anywhere from 600-115,...",0,"An ambition to succeed, A desire to be the bes...","Residual Income, Travel dollars, Car Dollars, ...",...,0,0,Other,Not Applicable,Unspecified,Marketing and Advertising,1,Marketing,1,1
17625,17626,manager,us,ky,customer,NaN,"This position is a full-time, salaried positio...",1,QUALIFICATIONS- 5 years of related experience ...,"- Full-time, salaried position- Potential for ...",...,0,0,Full-time,Associate,Professional,Financial Services,1,Administrative,1,1
17634,17635,urgent part timers & full timers required.,au,nsw,unspecified,NaN,URGENT Part Timers &amp; Full Timers Required....,0,NaN,NaN,...,0,0,Part-time,NaN,NaN,NaN,0,NaN,0,1
2444,2445,tech,us,ca,unspecified,NaN,1. Technical Lead - Rhomobile Technical Mob...,1,Requirement / JD: Drive the mobile app develop...,NaN,...,0,1,Contract,Mid-Senior level,Bachelor's Degree,Information Technology and Services,1,Information Technology,1,1
5849,5850,customer,us,ma,unspecified,NaN,2168Job Title: Product Insight Analyst Propert...,1,Bottom Line Requirements:1. Graduate degree i...,NaN,...,0,0,Full-time,NaN,NaN,NaN,0,NaN,0,1
7482,7483,customer,us,ky,unspecified,NaN,Apply using below link#URL_80c8ea5f0db2d88ff77...,0,NaN,NaN,...,0,0,Full-time,NaN,NaN,Hospital & Health Care,1,NaN,0,1
17807,17808,tech,us,al,unspecified,NaN,We're looking for self-motivated individuals w...,0,Must be able to work from home,NaN,...,1,0,Full-time,Entry level,High School or equivalent,Consumer Services,1,Customer Service,1,1
17645,17646,customer,us,,unspecified,NaN,Take calls at home for our ad company.We have ...,1,Computer with internet.Computer knowledge a mu...,No benefits.,...,0,1,Part-time,Not Applicable,Unspecified,Marketing and Advertising,1,Other,1,1
6992,6993,tech,us,tx,human resources,NaN,Tasks may include:Contribute to the continued ...,1,Position QualificationsWhat we're askingBachel...,NaN,...,0,0,Full-time,NaN,NaN,Oil & Energy,1,Human Resources,0,1


In [187]:
df.iloc[1217]['description']

'Colonial is looking for 5 sharp people to inform small business owners about the newest health benefits available for small business owners in illinois. \xa0If you are a great talker and dresser the rest is simple. \xa0Make 30 calls a day. \xa0Set appointments \xa0 \xa0and make presentations to earn over $1000 a week we pay 4 times a month. \xa0Its just that sim\\ple and we have been doing the same system for over 100 years to bring home the bacon. \xa0Training and leads provided. \xa0Insurance llicense preferred but we will provide a temporary if needed at no charge. \xa0Management available.'

I dont need job_id 

In [188]:
del df['job_id']

In [189]:
df.head()

,title,country,state,department,company_profile,description,has_reqs,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,has_industry,function,has_salary,fraudulent
0,marketing,us,ny,marketing,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",1,Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,0,Marketing,0,0
1,customer,nz,,success,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,1,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,1,Customer Service,0,0
2,commissioning machinery assistant (cma),us,ia,unspecified,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",1,Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,0,NaN,0,0
3,accounting,us,dc,sales,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,1,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,1,Sales,0,0
4,manager,us,fl,unspecified,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,1,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,1,Health Care Provider,0,0


In [190]:
df.nunique()

title                   3233
country                   91
state                    326
department               790
company_profile         1709
description            14801
has_reqs                   2
requirements           11968
benefits                6205
telecommuting              2
has_company_logo           2
has_questions              2
employment_type            5
required_experience        7
required_education        13
industry                 131
has_industry               2
function                  37
has_salary                 2
fraudulent                 2
dtype: int64

#### Important Decision

From this point on I am going to drop the non-numerical variables columns but country and state, since I won't be using any NLP for the scope of this project.

In [191]:
df.drop(columns=['title', 'department', 'company_profile', 'description', 'requirements', 'benefits', 'industry', 'function'], inplace=True)

In [192]:
df.nunique()

country                 91
state                  326
has_reqs                 2
telecommuting            2
has_company_logo         2
has_questions            2
employment_type          5
required_experience      7
required_education      13
has_industry             2
has_salary               2
fraudulent               2
dtype: int64

*Decision*  
  
I am now going to target encode the country and states in order to get more insight. 

In [193]:
ce_te = ce.TargetEncoder(cols=['country', 'state'])
ce_te_df = ce_te.fit_transform(df, df['fraudulent'])
ce_te_df

/Users/tinym/opt/anaconda3/lib/python3.9/site-packages/category_encoders/target_encoder.py:122: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
/Users/tinym/opt/anaconda3/lib/python3.9/site-packages/category_encoders/target_encoder.py:127: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."


,country,state,has_reqs,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,has_industry,has_salary,fraudulent
0,0.068506,0.054011,1,0,1,0,Other,Internship,NaN,0,0,0
1,0.000000,0.026636,1,0,1,0,Full-time,Not Applicable,NaN,1,0,0
2,0.068506,0.035294,1,0,1,0,NaN,NaN,NaN,0,0,0
3,0.068506,0.025362,1,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,1,0,0
4,0.068506,0.072289,1,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
17875,0.026258,0.029661,1,0,1,1,Full-time,Mid-Senior level,NaN,1,0,0
17876,0.068506,0.041139,1,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,1,0,0
17877,0.068506,0.155897,1,0,0,0,Full-time,NaN,NaN,0,0,0
17878,0.000006,0.073171,1,0,0,1,Contract,Not Applicable,Professional,1,0,0


In [194]:
ce_te_df.corr()

,country,state,has_reqs,telecommuting,has_company_logo,has_questions,has_industry,has_salary,fraudulent
country,1.000000,0.492433,0.004270,0.020063,-0.121482,-0.087434,0.005496,-0.001095,0.190541
state,0.492433,1.000000,-0.002176,0.011596,-0.104348,-0.071462,0.017576,0.002387,0.256793
has_reqs,0.004270,-0.002176,1.000000,-0.009678,0.137564,0.038413,0.060417,-0.006388,-0.081021
telecommuting,0.020063,0.011596,-0.009678,1.000000,-0.019836,0.020345,0.021237,0.071431,0.034523
has_company_logo,-0.121482,-0.104348,0.137564,-0.019836,1.000000,0.233932,0.137759,-0.058141,-0.261971
has_questions,-0.087434,-0.071462,0.038413,0.020345,0.233932,1.000000,0.055148,0.005103,-0.091627
has_industry,0.005496,0.017576,0.060417,0.021237,0.137759,0.055148,1.000000,0.223232,-0.021915
has_salary,-0.001095,0.002387,-0.006388,0.071431,-0.058141,0.005103,0.223232,1.000000,0.059696
fraudulent,0.190541,0.256793,-0.081021,0.034523,-0.261971,-0.091627,-0.021915,0.059696,1.000000


In [195]:
df.isna().sum()

country                   0
state                     0
has_reqs                  0
telecommuting             0
has_company_logo          0
has_questions             0
employment_type        3471
required_experience    7050
required_education     8105
has_industry              0
has_salary                0
fraudulent                0
dtype: int64

At this point also going to remove employment_type, required_experience and required_education due to the high number of NaNs.

In [196]:
df = ce_te_df

In [197]:
del df['employment_type']
del df['required_experience']
del df['required_education']

In [198]:
df.head()

,country,state,has_reqs,telecommuting,has_company_logo,has_questions,has_industry,has_salary,fraudulent
0,0.068506,0.054011,1,0,1,0,0,0,0
1,0.000000,0.026636,1,0,1,0,1,0,0
2,0.068506,0.035294,1,0,1,0,0,0,0
3,0.068506,0.025362,1,0,1,0,1,0,0
4,0.068506,0.072289,1,0,1,1,1,0,0


In [199]:
df.corr()

,country,state,has_reqs,telecommuting,has_company_logo,has_questions,has_industry,has_salary,fraudulent
country,1.000000,0.492433,0.004270,0.020063,-0.121482,-0.087434,0.005496,-0.001095,0.190541
state,0.492433,1.000000,-0.002176,0.011596,-0.104348,-0.071462,0.017576,0.002387,0.256793
has_reqs,0.004270,-0.002176,1.000000,-0.009678,0.137564,0.038413,0.060417,-0.006388,-0.081021
telecommuting,0.020063,0.011596,-0.009678,1.000000,-0.019836,0.020345,0.021237,0.071431,0.034523
has_company_logo,-0.121482,-0.104348,0.137564,-0.019836,1.000000,0.233932,0.137759,-0.058141,-0.261971
has_questions,-0.087434,-0.071462,0.038413,0.020345,0.233932,1.000000,0.055148,0.005103,-0.091627
has_industry,0.005496,0.017576,0.060417,0.021237,0.137759,0.055148,1.000000,0.223232,-0.021915
has_salary,-0.001095,0.002387,-0.006388,0.071431,-0.058141,0.005103,0.223232,1.000000,0.059696
fraudulent,0.190541,0.256793,-0.081021,0.034523,-0.261971,-0.091627,-0.021915,0.059696,1.000000


### Proceeding with a Tree and RF model